In [ ]:
# import dependencies
import pandas as pd
import os

import sqlalchemy
from sqlalchemy import create_engine, MetaData
import pymysql
pymysql.install_as_MySQLdb()

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [3]:
!ls info_data

asylum_yr16.csv
battle_deaths_by_country.csv
battle_deaths_yr16.csv
countries.csv
country_info.csv
data_cleaning_for_info_table.ipynb
gdp.csv
gdpClean.csv
origin_yr16.csv
POP.csv
population.csv
refugees_by_country_of_asylum.csv
refugees_by_country_of_origin.csv


In [ ]:
# load data into pandas dataframes
country_info_file = os.path.join("info_data", "country_info.csv")

country_info_data = pd.read_csv(country_info_file)

In [22]:
# check for null values and replace
country_info_data = country_info_data.replace(to_replace="..", value=0)

In [23]:
country_info_data.head()

,Country Name,GDP [YR2015],Population [YR2016],Asylum [YR2016],Origin [YR2016]
0,Afghanistan,19215562178.9798,"34,656",17980,2501410
1,Albania,11335264966.561,"2,876",0,11050
2,Algeria,165874330876.321,"40,606",86,3675
3,American Samoa,659000000,56,0,0
4,Andorra,2811489408.89431,77,0,0


In [32]:
column_names = ["country_name","gdp_YR2015", "population_YR2016", "asylum_YR2016", "origin_YR2016"]

country_info_data.columns = column_names

In [33]:
# create database
connection_string = "sqlite:///countries_info.sqlite"
engine = create_engine(connection_string)
conn = engine.connect()

In [35]:
# create ORM classes for each table
class CountryInfo(Base):
    __tablename__ = 'info_table'

    id = Column(Integer, primary_key=True)
    country_name = Column(Text)
    gdp_YR2015 = Column(Float)
    population_YR2016 = Column(Float)
    asylum_YR2016 = Column(Float)
    origin_YR2016 = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

C:\Users\mrbal\Anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.CountryInfo, and will be replaced in the string-lookup table.
  item.__name__


In [36]:
CountryInfo.__table__.drop(engine)

OperationalError: (sqlite3.OperationalError) no such table: info_table1 [SQL: '\nDROP TABLE info_table1']

In [20]:
# write data to tables
country_info_data_to_write = country_info_data.to_dict(orient='records')

metadata = MetaData(bind=engine)
metadata.reflect()

country_info_table = sqlalchemy.Table('info_table', metadata, autoload=True)
conn.execute(country_info_table.insert(), country_info_data_to_write)

NoSuchTableError: info_table

In [43]:
# confirm data properly written to table

battle_deaths_head = conn.execute("select * from battle_deaths limit 5").fetchall()
asylum_head = conn.execute("select * from asylum limit 5").fetchall()
origin_head = conn.execute("select * from origin limit 5").fetchall()

print("First 5 rows of battle_deaths table:")
print(battle_deaths_head)
print("First 5 rows of asylum table:")
print(asylum_head)
print("First 5 rows of origin table:")
print(origin_head)

First 5 rows of battle_deaths table:
[(1, 'Afghanistan', 'AFG', 1478, 3302, 4276, 4071, 8937, 5499, 3177, 6396, 6256, 4629, 5235, 5148, 890, 687, 715, 1595, 4750, 6906, 5552, 6341, 6864, 7405, 7719, 8056, 12285, 17273, 17980, 0), (2, 'Albania', 'ALB', 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (3, 'Algeria', 'DZA', 0, 34, 257, 800, 1916, 3008, 1084, 1403, 3029, 1016, 931, 642, 538, 670, 397, 217, 255, 472, 339, 505, 236, 267, 256, 147, 107, 110, 86, 0), (4, 'American Samoa', 'ASM', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (5, 'Andorra', 'AND', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]
First 5 rows of asylum table:
[(1, 'Afghanistan', 'AFG', 50, 38, 60025, 32132, 19131, 19605, 18775, 5, 0, 0, 0, 6, 3, 22, 30, 32, 35, 42, 37, 37, 6434, 3009, 16187, 16863, 300423, 257554, 59770, 0), (2, 'Albania', 'ALB', 0, 0, 3000, 3000, 3000, 4720, 4925, 30, 22332, 3930, 523, 

In [44]:
asylum_data.head()

,country_name,country_lat,country_lon,country_code,y1990,y1991,y1992,y1993,y1994,y1995,...,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,Albania,41.0000,20.0,ALB,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Algeria,28.0000,3.0,DZA,0,34,257,800,1916,3008,...,339,505,236,267,256,147,107,110,86,0
2,American Samoa,-14.3333,-170.0,ASM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Andorra,42.5000,1.6,AND,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,-12.5000,18.5,AGO,2558,526,2321,12054,3974,162,...,0,25,0,0,0,0,0,0,0,0


In [45]:
origin_data.head()

,country_name,country_lat,country_lon,country_code,y1990,y1991,y1992,y1993,y1994,y1995,...,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,Albania,41.0000,20.0,ALB,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Algeria,28.0000,3.0,DZA,0,34,257,800,1916,3008,...,339,505,236,267,256,147,107,110,86,0
2,American Samoa,-14.3333,-170.0,ASM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Andorra,42.5000,1.6,AND,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,-12.5000,18.5,AGO,2558,526,2321,12054,3974,162,...,0,25,0,0,0,0,0,0,0,0


In [47]:
battle_deaths_data.head()

,country_name,country_lat,country_lon,country_code,y1990,y1991,y1992,y1993,y1994,y1995,...,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,Albania,41.0000,20.0,ALB,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Algeria,28.0000,3.0,DZA,0,34,257,800,1916,3008,...,339,505,236,267,256,147,107,110,86,0
2,American Samoa,-14.3333,-170.0,ASM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Andorra,42.5000,1.6,AND,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,-12.5000,18.5,AGO,2558,526,2321,12054,3974,162,...,0,25,0,0,0,0,0,0,0,0
